# Notebook 3: Automating Repetitive Tasks

## LLM Allies for Scientists Workshop

**Duration**: ~35 minutes

**Learning Goals**:
- Use LLMs to generate data processing scripts
- Batch process multiple files
- Create reusable analysis pipelines
- Clean messy data with AI assistance

---

## 1. Setup

In [ ]:
# Install required packages
!pip install openai pandas numpy -q

In [ ]:
import os
import re
import openai
import pandas as pd
import numpy as np
from pathlib import Path

# API setup
try:
    from google.colab import userdata
    api_key = userdata.get('OPENAI_API_KEY')
except:
    api_key = os.environ.get('OPENAI_API_KEY')

if not api_key:
    raise ValueError("Please set your OPENAI_API_KEY")

client = openai.OpenAI(api_key=api_key)
print("Setup complete!")

In [ ]:
# Helper functions from previous notebooks
def ask_llm(system_prompt, user_message, model="gpt-4o-mini", temperature=0.7):
    """Simple wrapper for OpenAI API calls."""
    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_message}
        ],
        temperature=temperature
    )
    return response.choices[0].message.content

def safe_execute(code_string, local_vars=None):
    """Execute code with error handling."""
    if local_vars is None:
        local_vars = {}
    
    # Clean markdown formatting
    clean_code = code_string.strip()
    if clean_code.startswith('```python'):
        clean_code = clean_code[9:]
    if clean_code.startswith('```'):
        clean_code = clean_code[3:]
    if clean_code.endswith('```'):
        clean_code = clean_code[:-3]
    clean_code = clean_code.strip()
    
    try:
        exec(clean_code, globals(), local_vars)
        return True, local_vars, None
    except Exception as e:
        return False, local_vars, str(e)

---

## 2. From Manual to Automated

### The "Show Me One, Automate the Rest" Pattern

The most powerful way to use LLMs for automation:
1. **Do one example manually** (or describe it in detail)
2. **Show the LLM what you did**
3. **Ask it to generalize into a reusable function**

In [ ]:
# System prompt for automation tasks
AUTOMATION_SYSTEM_PROMPT = """You are an expert Python programmer who specializes in data processing automation.

Rules:
1. Write clean, well-documented Python code
2. Include docstrings with clear parameter descriptions
3. Add appropriate error handling
4. Use type hints where helpful
5. Output only code, no explanations unless asked
6. Prefer pandas for data manipulation
7. Make functions reusable and generalizable
"""

### Example: Automating a Manual Process

Let's say you manually cleaned some concentration values. Now let's automate it.

In [ ]:
# Create messy data (simulating real-world issues)
messy_df = pd.DataFrame({
    'sample_id': ['EXP001', 'EXP002', 'EXP003', 'EXP004', 'EXP005', 'EXP006'],
    'concentration': ['1.5', '2.0 mM', '1.5mM', 'N/A', '3.0', '2.5 mM'],
    'measurement_date': ['2024-01-15', '01/16/2024', 'Jan 17, 2024', '2024-01-18', '2024/01/19', '20-Jan-2024'],
    'temperature': ['25C', '25.0', '25 C', 'NA', '25.5°C', '']
})

print("Messy data:")
print(messy_df)

In [ ]:
# Describe the problem to the LLM
cleaning_request = """
I have CSV files where the 'concentration' column sometimes has:
- Numbers like "1.5"
- Text like "1.5 mM" or "1.5mM"
- Missing values as "N/A", "NA", or empty

Write a function called `clean_concentration` that:
1. Takes a pandas Series as input
2. Extracts the numeric value (assuming all values are in mM)
3. Returns NaN for missing/invalid values
4. Returns a cleaned pandas Series

Include examples in the docstring.
"""

code = ask_llm(AUTOMATION_SYSTEM_PROMPT, cleaning_request, temperature=0.3)
print(code)

In [ ]:
# Execute and test the generated function
success, local_vars, error = safe_execute(code)

if success:
    # The function should now be available
    print("Original concentration values:")
    print(messy_df['concentration'].tolist())
    print("\nCleaned values:")
    print(clean_concentration(messy_df['concentration']).tolist())
else:
    print(f"Error: {error}")

---

## 3. File Processing Workflows

### 3.1 Creating Sample Files

Let's create some sample files to work with.

In [ ]:
# Create a directory with sample experiment files
import os

# Create directory
os.makedirs('sample_experiments', exist_ok=True)

# Generate sample experiment files with different formats
experiments = [
    {
        'filename': 'sample_experiments/exp_2024_01.csv',
        'data': pd.DataFrame({
            'SubjectID': ['M001', 'M002', 'M003', 'M004', 'M005'],
            'Treatment': ['Drug_A', 'Drug_A', 'Placebo', 'Drug_A', 'Placebo'],
            'Measurement': [45.2, 52.1, 32.4, 48.7, 33.8],
            'Date': ['2024-01-15']*5
        })
    },
    {
        'filename': 'sample_experiments/exp_2024_02.csv',
        'data': pd.DataFrame({
            'subject_id': ['M006', 'M007', 'M008', 'M009', 'M010'],
            'treatment': ['Drug_B', 'Placebo', 'Drug_B', 'Drug_B', 'Placebo'],
            'measurement_value': [61.2, 31.5, 58.9, 62.4, 34.1],
            'exp_date': ['2024-02-10']*5
        })
    },
    {
        'filename': 'sample_experiments/exp_2024_03.csv',
        'data': pd.DataFrame({
            'ID': ['M011', 'M012', 'M013', 'M014', 'M015'],
            'Trt': ['Drug_A', 'Drug_B', 'Control', 'Drug_A', 'Drug_B'],
            'Value': [47.3, 59.8, 32.9, 51.2, 63.1],
            'Date_Collected': ['2024-03-05']*5
        })
    }
]

for exp in experiments:
    exp['data'].to_csv(exp['filename'], index=False)
    print(f"Created: {exp['filename']}")
    print(exp['data'].head(2))
    print()

### 3.2 Batch Processing Multiple Files

In [ ]:
# Show the LLM what we're working with
file_descriptions = """
I have 3 CSV files from different experiments. They have similar data but different column names:

File 1 (exp_2024_01.csv):
- SubjectID, Treatment, Measurement, Date

File 2 (exp_2024_02.csv):
- subject_id, treatment, measurement_value, exp_date

File 3 (exp_2024_03.csv):
- ID, Trt, Value, Date_Collected
- Note: 'Trt' values include 'Control' instead of 'Placebo'

I want to combine them into a single DataFrame with standardized columns:
- subject_id
- treatment (with 'Control' renamed to 'Placebo')
- measurement
- date
- source_file (which file the data came from)

Write a function that:
1. Takes a directory path as input
2. Finds all CSV files
3. Standardizes column names using the mapping I described
4. Combines all files into one DataFrame
5. Returns the combined DataFrame
"""

code = ask_llm(AUTOMATION_SYSTEM_PROMPT, file_descriptions, temperature=0.3)
print(code)

In [ ]:
# Execute and test
success, local_vars, error = safe_execute(code)

if success:
    # Try the function
    combined_df = combine_experiment_files('sample_experiments')
    print(f"Combined {len(combined_df)} rows from experiment files:")
    print(combined_df)
else:
    print(f"Error: {error}")

---

## 4. Data Cleaning with AI

### 4.1 Describe Messy Data, Get Cleaning Code

In [ ]:
# Create messy data with multiple issues
messy_data = pd.DataFrame({
    'sample_id': ['EXP001', 'EXP002', 'EXP003', 'EXP004', 'EXP005', 
                  'EXP006', 'EXP007', 'EXP008', 'EXP009', 'EXP010'],
    'concentration': ['1.5', '2.0 mM', '1.5mM', 'N/A', '3.0', 
                      '2.5 mM', '1.0', 'NA', '1.75 mM', ''],
    'measurement_date': ['2024-01-15', '01/16/2024', 'Jan 17, 2024', '2024-01-18', '2024/01/19',
                         '20-Jan-2024', '2024-01-21', 'January 22, 2024', '2024-01-23', '01-24-2024'],
    'temperature': ['25C', '25.0', '25 C', 'NA', '25.5°C', 
                    '', '25C', '25.0 C', '25', 'N/A'],
    'cell_count': ['1.2e6', '1500000', '1.1E6', '1.3e6', '1250000',
                   '1.4e6', 'N/A', '1.35E6', '1.28e6', '1320000'],
    'status': ['Complete', 'complete', 'COMPLETE', 'Failed', 'Complete',
               'complete', 'Complete', 'COMPLETE', 'Incomplete', 'Complete']
})

print("Messy data sample:")
print(messy_data)

In [ ]:
# Ask LLM to analyze and clean the data
cleaning_prompt = f"""
Analyze this messy data and write a comprehensive cleaning function:

{messy_data.to_string()}

Issues I've noticed:
1. concentration: mixed formats (plain numbers, with units, missing values)
2. measurement_date: many different date formats
3. temperature: mixed formats (with/without units, missing values)
4. cell_count: scientific notation mixed with regular numbers, missing values
5. status: inconsistent capitalization

Write a function `clean_experiment_data(df)` that:
1. Cleans each column appropriately
2. Converts dates to datetime
3. Converts numeric columns to float
4. Standardizes status to title case
5. Returns the cleaned DataFrame

Include a summary of what was cleaned at the end.
"""

code = ask_llm(AUTOMATION_SYSTEM_PROMPT, cleaning_prompt, temperature=0.3)
print(code)

In [ ]:
# Execute and test
success, local_vars, error = safe_execute(code)

if success:
    cleaned_df = clean_experiment_data(messy_data)
    print("\nCleaned data:")
    print(cleaned_df)
    print("\nData types:")
    print(cleaned_df.dtypes)
else:
    print(f"Error: {error}")

---

## 5. Practical Exercises

### Exercise A: Clean Inconsistent Date Formats

In [ ]:
# Sample dates in various formats
dates_df = pd.DataFrame({
    'entry': range(1, 11),
    'recorded_date': [
        '2024-01-15',
        '01/16/2024', 
        'Jan 17, 2024',
        '18-01-2024',
        '2024/01/19',
        '20 January 2024',
        '1/21/24',
        '22-Jan-2024',
        'January 23rd, 2024',
        '24.01.2024'
    ]
})

print("Dates to clean:")
print(dates_df)

# YOUR CODE HERE: Write a prompt to create a date cleaning function
# date_cleaning_prompt = "..."
# code = ask_llm(AUTOMATION_SYSTEM_PROMPT, date_cleaning_prompt, temperature=0.3)
# print(code)

### Exercise B: Extract Numerical Values from Text

In [ ]:
# Lab notes with embedded numerical values
lab_notes_df = pd.DataFrame({
    'experiment': ['EXP001', 'EXP002', 'EXP003', 'EXP004', 'EXP005'],
    'notes': [
        'Measured pH of 7.2, temperature stable at 37C',
        'Final concentration: 2.5 mM. Yield was 89%.',
        'Cell viability 95.3%, counted 1.2e6 cells',
        'OD600 reading: 0.85. Growth rate 0.42/hr.',
        'Protein concentration 150 ug/mL (Bradford assay)'
    ]
})

print("Lab notes to parse:")
print(lab_notes_df)

# YOUR CODE HERE: Write a prompt to extract numerical values
# The output should have columns for each type of measurement found

### Exercise C: Merge Files with Different Schemas

In [ ]:
# Create files with different schemas but overlapping data
os.makedirs('merge_exercise', exist_ok=True)

# File 1: Subject demographics
demographics = pd.DataFrame({
    'participant_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'age': [28, 34, 45, 31, 52],
    'gender': ['F', 'M', 'F', 'M', 'F'],
    'group': ['Treatment', 'Control', 'Treatment', 'Control', 'Treatment']
})
demographics.to_csv('merge_exercise/demographics.csv', index=False)

# File 2: Baseline measurements
baseline = pd.DataFrame({
    'SubjectID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'BaselineScore': [72, 68, 81, 75, 69],
    'BaselineDate': ['2024-01-01']*5
})
baseline.to_csv('merge_exercise/baseline_scores.csv', index=False)

# File 3: Follow-up measurements
followup = pd.DataFrame({
    'ID': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'Score_Week4': [78, 70, 89, 74, 82],
    'Score_Week8': [82, 69, 94, 76, 88],
    'Completed': [True, True, True, False, True]
})
followup.to_csv('merge_exercise/followup.csv', index=False)

print("Files created. Schema overview:")
print("\ndemographics.csv:", demographics.columns.tolist())
print("baseline_scores.csv:", baseline.columns.tolist())
print("followup.csv:", followup.columns.tolist())

# YOUR CODE HERE: Write a prompt to merge these files
# Hint: The subject ID column has different names in each file

---

## 6. Building Your Script Library

### 6.1 Saving Functions for Reuse

In [ ]:
def save_function_to_file(code, filename, module_docstring=None):
    """
    Save generated code to a Python file for reuse.
    
    Args:
        code: Generated Python code
        filename: Output file name (e.g., 'cleaning_utils.py')
        module_docstring: Optional docstring for the module
    """
    # Clean the code
    clean_code = code.strip()
    if clean_code.startswith('```python'):
        clean_code = clean_code[9:]
    if clean_code.startswith('```'):
        clean_code = clean_code[3:]
    if clean_code.endswith('```'):
        clean_code = clean_code[:-3]
    clean_code = clean_code.strip()
    
    # Add imports and docstring
    header = '"""\n' + (module_docstring or 'Auto-generated utility functions.') + '\n"""\n\n'
    header += 'import pandas as pd\nimport numpy as np\nimport re\nfrom datetime import datetime\n\n'
    
    with open(filename, 'w') as f:
        f.write(header + clean_code)
    
    print(f"Saved to {filename}")

# Example: Save a cleaning function
# save_function_to_file(code, 'my_cleaning_utils.py', 'Data cleaning utilities for my lab')

### 6.2 Generate Documentation

In [ ]:
def add_docstring(code):
    """
    Ask LLM to add or improve docstrings in code.
    """
    prompt = f"""Add comprehensive docstrings to this code.
Include:
- Function description
- Args with types and descriptions
- Returns description
- Example usage
- Any important notes or warnings

Code:
{code}

Output only the improved code with docstrings."""
    
    return ask_llm(AUTOMATION_SYSTEM_PROMPT, prompt, temperature=0.3)

# Example: Improve documentation on generated code
simple_function = """
def normalize_column(series):
    return (series - series.min()) / (series.max() - series.min())
"""

documented = add_docstring(simple_function)
print(documented)

---

## 7. Advanced: Pipeline Generator

Create a complete data processing pipeline from a description.

In [ ]:
def generate_pipeline(description, input_example=None):
    """
    Generate a complete data processing pipeline from a description.
    
    Args:
        description: Natural language description of the pipeline
        input_example: Optional sample input data (as string)
    """
    pipeline_prompt = f"""Create a complete data processing pipeline based on this description:

{description}

"""
    if input_example:
        pipeline_prompt += f"\nExample input data:\n{input_example}\n"
    
    pipeline_prompt += """
Create a class called DataPipeline with:
1. __init__ method that sets up any configuration
2. Individual methods for each processing step
3. A run() method that executes all steps in order
4. A save_results() method to export the final data
5. Logging at each step to show progress

Include example usage at the end.
"""
    
    return ask_llm(AUTOMATION_SYSTEM_PROMPT, pipeline_prompt, temperature=0.3)

In [ ]:
# Generate a pipeline
pipeline_description = """
I need a pipeline to process gene expression data that:
1. Loads a CSV file with gene names in the first column and sample values in other columns
2. Filters out genes with low variance (bottom 20%)
3. Log2-transforms the expression values (adding 1 to avoid log of zero)
4. Z-score normalizes each gene across samples
5. Identifies the top 50 most variable genes
6. Saves the processed data to a new CSV file
"""

pipeline_code = generate_pipeline(pipeline_description)
print(pipeline_code)

In [ ]:
# Create sample gene expression data to test
np.random.seed(42)
n_genes = 100
n_samples = 6

gene_data = pd.DataFrame({
    'gene_name': [f'Gene_{i:03d}' for i in range(n_genes)]
})

for i in range(n_samples):
    gene_data[f'Sample_{i+1}'] = np.random.exponential(scale=100, size=n_genes)

gene_data.to_csv('gene_expression.csv', index=False)
print("Sample gene expression data:")
print(gene_data.head())

In [ ]:
# Execute and test the pipeline
success, local_vars, error = safe_execute(pipeline_code)

if success:
    print("Pipeline created successfully!")
    # Try to run it
    try:
        pipeline = DataPipeline('gene_expression.csv')
        result = pipeline.run()
        print("\nPipeline output:")
        print(result.head())
    except Exception as e:
        print(f"Pipeline execution error: {e}")
else:
    print(f"Error creating pipeline: {error}")

---

## 8. Key Takeaways

### What You've Learned:
1. **Show and Automate**: Describe manual processes to get automated functions
2. **Batch Processing**: Handle multiple files with different formats
3. **Data Cleaning**: Let AI write cleaning code from problem descriptions
4. **Pipeline Generation**: Create complete processing workflows

### Best Practices:
- **Be specific** about edge cases and data quirks
- **Include examples** of messy data in your prompts
- **Test generated code** on sample data before using on real data
- **Save useful functions** for reuse across projects
- **Add documentation** to generated code

### Prompt Patterns for Automation:

| Task | Prompt Pattern |
|------|----------------|
| Clean column | "Column has [formats]. Write function to extract [what you need]." |
| Merge files | "Files have columns [list]. Create unified schema with [columns]." |
| Batch process | "For each file in [dir], do [action] and combine results." |
| Pipeline | "Create pipeline that: 1) [step], 2) [step], ... n) [step]." |

---

## Next: Notebook 4 - Your First AI Agent

In the next notebook, you'll learn to:
- Understand what agents are and when to use them
- Build a simple research assistant agent
- Chain multiple LLM calls for complex tasks

In [ ]:
# Cleanup (optional)
import shutil
# Uncomment to clean up created directories
# shutil.rmtree('sample_experiments', ignore_errors=True)
# shutil.rmtree('merge_exercise', ignore_errors=True)
# os.remove('gene_expression.csv') if os.path.exists('gene_expression.csv') else None